In [1]:
%tensorflow_version 2.x
!pip install bert-for-tf2
!pip install sentencepiece
!pip install tensorflow-addons

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
# multi-label hamming loss
import tensorflow_addons as tfa
import bert
from tensorflow.keras.models import  Model
from tqdm import tqdm
import numpy as np
from collections import namedtuple
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from google.colab import files
import os

uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ="/content/gcs.json"
os.environ['TFHUB_CACHE_DIR'] ="gs://my-data-bucket-colab/toxic-comments-classification/model"
# To successfully train on Colab TPU, you have to grant anonymous read permission to GCS buckets

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Saving gcs.json to gcs.json
User uploaded file "gcs.json" with length 2297 bytes


In [3]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Initializing the TPU system: grpc://10.64.230.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.64.230.210:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [0]:

def results(pred_results,ground_truth,threshold):
    pred_results = (pred_results>=threshold).astype(int)
    print("Positive labels RECALL-thre.{}:{}%".format(threshold,100*np.sum(pred_results*ground_truth)/np.sum(ground_truth)))
    cf = tfa.metrics.MultiLabelConfusionMatrix(num_classes=6)
    cf.update_state(ground_truth, pred_results)
    print('Confusion matrix:', cf.result().numpy())
    hl = tfa.metrics.HammingLoss(mode='multilabel', threshold=threshold)
    hl.update_state(ground_truth, pred_results)
    print('Hamming loss:', hl.result().numpy())

def _floats_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def parse_func(s_example):
    features = {
                'labels':tf.io.FixedLenFeature((6,),tf.float32),
                'masks':tf.io.FixedLenFeature((128,),tf.int64),
                'ids':tf.io.FixedLenFeature((128,),tf.int64),
                'segments':tf.io.FixedLenFeature((128,),tf.int64),
                }
    example = tf.io.parse_single_example(s_example, features=features)
    return (tf.cast(example['ids'],tf.int32),tf.cast(example['masks'],tf.int32),tf.cast(example['segments'],tf.int32)),example['labels']
def imgSerialization (ids,masks,segments,labels):
    example = tf.train.Example(features=tf.train.Features(feature={
        'labels':_floats_feature(labels),
        'ids':_int64_feature(ids),
        'masks':_int64_feature(masks),
        'segments':_int64_feature(segments)
        }))
    return example.SerializeToString()


In [0]:
def build_model_compile():
    hl = tfa.metrics.HammingLoss(mode='multilabel', threshold=0.4)
    bert_layer=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=False)
    MAX_SEQ_LEN=128
    input_word_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                          name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                      name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32,
                                        name="segment_ids")
    @tf.function
    def custom_recall(y_true,y_pred):
        y_true = tf.convert_to_tensor(y_true)
        Y_PRED = tf.cast(y_pred>=0.4,tf.float32)
        recall = tf.math.divide_no_nan(tf.math.reduce_sum(tf.math.multiply(y_true,Y_PRED)),tf.math.reduce_sum(y_true))
        return recall
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    FullTokenizer=bert.bert_tokenization.FullTokenizer
    vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case=bert_layer.resolved_object.do_lower_case.numpy()
    tokenizer=FullTokenizer(vocab_file,do_lower_case)
    x = tf.keras.layers.Dropout(0.1)(pooled_output)
    x= tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    out = tf.keras.layers.Dense(6, activation="sigmoid", name="dense_output")(x)

    model = tf.keras.models.Model(
          inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=[hl,custom_recall])

    return model

In [6]:
with tpu_strategy.scope():
  model = build_model_compile()

#bert_layer,model = build_model_compile()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [7]:
    
BATCH_SIZE=256
dataset = tf.data.TFRecordDataset('gs://my-data-bucket-colab/toxic-comments-classification/train.tfrecords').map(parse_func)
train_dataset = dataset.repeat().shuffle(2000).batch(BATCH_SIZE,drop_remainder=True)
model.fit(train_dataset,epochs=3, steps_per_epoch=600000//BATCH_SIZE)
#model.save_weights('/content/chkpt')

Epoch 1/3
2343/2343 [==============================] - 132s 56ms/step - custom_recall: 0.7757 - hamming_loss: 0.1361 - loss: 0.2987
Epoch 2/3
2343/2343 [==============================] - 132s 56ms/step - custom_recall: 0.8100 - hamming_loss: 0.1238 - loss: 0.2725
Epoch 3/3
2343/2343 [==============================] - 133s 57ms/step - custom_recall: 0.8182 - hamming_loss: 0.1202 - loss: 0.2648


In [8]:
test_dataset= tf.data.TFRecordDataset('gs://my-data-bucket-colab/toxic-comments-classification/test.tfrecords').map(parse_func).batch(1024,drop_remainder=True)
pred_results= model.predict(test_dataset)
ground_truth= next(iter(test_dataset.unbatch().batch(40000).map(lambda a, b: b))).numpy().astype(int)
results(pred_results,ground_truth,.4)

Positive labels RECALL-thre.0.4:80.18147774737146%
Confusion matrix: [[[25959.  2763.]
  [  470.  2552.]]

 [[30991.   434.]
  [  119.   200.]]

 [[28507.  1567.]
  [  313.  1357.]]

 [[31461.   190.]
  [   45.    48.]]

 [[28498.  1679.]
  [  304.  1263.]]

 [[31019.   453.]
  [  125.   147.]]]
Hamming loss: 0.04442835


In [0]:
model.save_weights('gs://my-data-bucket-colab/content/chkpt')